In [1]:
## SIMULATION PARAMETERS
verbosity         = 8        # Set debug print statement verbosity level (0 = Standard, -1 = Off)
use_mass_units    = True     # Toggle whether calculations / results are given in units of pi-axion mass (True) or eV (False)
use_natural_units = True     # Toggle whether calculations / results are given in c = h = G = 1 (True) or SI units (False)   || NOTE: full SI/phsyical unit support is still WIP!!
save_output_files = True     # Toggle whether or not the results from this notebook run are written to a data directory
save_plots        = False    # Toggle whether or not to generate and save plots
use_logsumexp     = False    # Toggle whether or not numerical integrations are performed in log-scale number space (increases computation time)

## CONFIG PARAMETERS
config_name = "piaxi_debug"   # Descriptive name for the given parameter case. Output files will be saved in a directory with this name.
seed = None
#seed = 269934808235809364794393739371438879116                      # rng_seed, integer value (None for random)
#seed = 191281265662032962853842089260008419904
num_cores = 48               # Number of parallel threads available
density = "1e20"              # Local DM energy density (in [Gev/cm^3])
method = "RK45"               # numerical integration method, to be passed to scipy.solve_ivp()

## Scenario to run
# NOTE: (NAME) indicates NAME is an alias of the previously listed item.
scenario = "SU6"           # one of: SINGLE, QCD, (AXION), SIMPLE, FULL, SU3, SU6, (SAMPLED), REALS, NEUTRALS, COMPLEX, CHARGED

In [2]:
#%cd ~/projects/pi-axiverse/

In [3]:
#%load piaxiverse.py

In [4]:
#%module load miniconda3/23.11.0s
#%module load texlive
#%source /oscar/runtime/software/external/miniconda3/23.11.0/etc/profile.d/conda.sh
#%conda activate piaxiverse

In [5]:
%%bash -s {density} {config_name} {seed} {scenario} {verbosity} {use_mass_units} {method} {save_output_files} {save_plots} {use_logsumexp}

#module load miniconda3/23.11.0s
module load texlive
#source /oscar/runtime/software/external/miniconda3/23.11.0/etc/profile.d/conda.sh
#conda activate piaxiverse

INPUT_VERBOSITY="${5:-1}"
PIAXI_VERBOSITY="${INPUT_VERBOSITY}"
if [[ "${PIAXI_VERBOSITY}" -gt "6" ]]
then
    conda info
fi

SCAN_ALL=0
if (( $SCAN_ALL ))
then
    N_SAMPLES=3
else
    N_SAMPLES=1
fi

# Toggle whether calculations are performed in natural units or physical units
INPUT_M_UNITS="${6:-True}"
if [[ "${INPUT_M_UNITS}" == "False" ]]
then
    PIAXI_UNITS="--no-use_mass_units --no-use_natural_units"
else
    PIAXI_UNITS="--use_mass_units --use_natural_units"
fi

# RNG seed
INPUT_SEED="${3:-None}"
if [[ "${INPUT_SEED}" == "None" ]]
then
    PIAXI_SEED=""
else
    PIAXI_SEED="--seed ${INPUT_SEED}"
fi

# Determine number of surviving pi-axion species and parameter distributions
INPUT_SCENARIO="${4:-0}"
PIAXI_JOB_SUFFIX=""
PIAXI_FIT=""
PIAXI_MASKS=""
if [[ "${INPUT_SCENARIO}" == "0" ]]
then
    PIAXI_DQMC="1 1 1 0 0 0"
elif [[ "${INPUT_SCENARIO}" == "SINGLE" ]]
then
    PIAXI_DQMC="0.5 0.5 0 0 0 0"
    PIAXI_JOB_SUFFIX="_single"
elif [[ "${INPUT_SCENARIO}" == "QCD" ]] || [[ "${INPUT_SCENARIO}" == "AXION" ]]
then
    PIAXI_DQMC="0.5 0.5 0 0 0 0"
    PIAXI_JOB_SUFFIX="_qcd"
    PIAXI_FIT="--fit_QCD"
    PIAXI_MASKS="--mask_complex --mask_charged"
elif [[ "${INPUT_SCENARIO}" == "SIMPLE" ]]
then
    PIAXI_DQMC="1 1 1 0 0 0"
    PIAXI_JOB_SUFFIX="_simple"
elif [[ "${INPUT_SCENARIO}" == "FULL" ]]
then
    PIAXI_DQMC="1 1 1 1 1 1"
    PIAXI_JOB_SUFFIX="_full"
elif [[ "${INPUT_SCENARIO}" == "SU3" ]]
then
    PIAXI_DQMC="x x x 0 0 0"
    PIAXI_JOB_SUFFIX="_SU3"
elif [[ "${INPUT_SCENARIO}" == "SAMPLED" ]] || [[ "${INPUT_SCENARIO}" == "SU6" ]]
then
    PIAXI_DQMC="x x x x x x"
    PIAXI_JOB_SUFFIX="_SU6"
elif [[ "${INPUT_SCENARIO}" == "REALS" ]] || [[ "${INPUT_SCENARIO}" == "AXIVERSE" ]]
then
    PIAXI_DQMC="x x x x x x"
    PIAXI_JOB_SUFFIX="_reals"
    PIAXI_MASKS="--mask_complex --mask_charged"
elif [[ "${INPUT_SCENARIO}" == "NEUTRALS" ]]
then
    PIAXI_DQMC="x x x x x x"
    PIAXI_JOB_SUFFIX="_neutrals"
    PIAXI_MASKS="--mask_charged"
elif [[ "${INPUT_SCENARIO}" == "COMPLEX" ]]
then
    PIAXI_DQMC="x x x x x x"
    PIAXI_JOB_SUFFIX="_complex"
    PIAXI_MASKS="--mask_reals"
elif [[ "${INPUT_SCENARIO}" == "COMPLEX_NEUTRALS" ]]
then
    PIAXI_DQMC="x x x x x x"
    PIAXI_JOB_SUFFIX="_complex_neutrals"
    PIAXI_MASKS="--mask_reals --mask_charged"
elif [[ "${INPUT_SCENARIO}" == "CHARGED" ]]
then
    PIAXI_DQMC="x x x x x x"
    PIAXI_JOB_SUFFIX="_charged"
    PIAXI_MASKS="--mask_reals --mask_complex"
fi

# Performance variables and SLURM environment configuration
INPUT_SYS_NAME="${2:-pi_axiverse}"
PIAXI_SYS_NAME="${INPUT_SYS_NAME}${PIAXI_JOB_SUFFIX}"
PIAXI_N_CORES="${SLURM_JOB_CPUS_PER_NODE}"
PIAXI_N_NODES="${SLURM_JOB_NUM_NODES}"
PIAXI_COREMEM="${SLURM_MEM_PER_NODE}"
PIAXI_JOB_QOS="${SLURM_JOB_QOS}"

# Whether or not to save output files and plots
INPUT_SAVE="${8:-True}"
INPUT_PLOTS="${9:-False}"
if [[ "${INPUT_PLOTS}" == "True" ]]
then
    PIAXI_PLOTS="--make_plots --no-show_plots"
else
    PIAXI_PLOTS="--no-make_plots --no-show_plots"
fi

if [[ "${INPUT_SAVE}" == "True" ]]
then
    PIAXI_SAVE="--save_output_files ${PIAXI_PLOTS}"
else
    PIAXI_SAVE="--no-save_output_files --no-make_plots --no-show_plots"
fi

# Number of timesteps
PIAXI_N_TIMES=300
# Max time value
PIAXI_T_MAX=30
# Number of k_modes
PIAXI_N_KMODE=200
# k_mode step-size
PIAXI_K_RES=0.1

PIAXI_RES_ARGS="--t ${PIAXI_T_MAX} --tN ${PIAXI_N_TIMES} --kN ${PIAXI_N_KMODE} --k_res ${PIAXI_K_RES}"

# || For QCD axion case:
# || Expect resonance for cases where [sqrt(2*Rho)/m_a] >= [F_pi]
# ||   --> let m_a = 1e-6 eV
# ||   --> let g_a = 1e-10 GeV^-1
# || Then: expect critical threshold between rho ~ 1e17 GeV and 1e18 GeV

# Density [GeV] || For QCD axion case ~ (amp_a)^2*m_a / 2
if (( $SCAN_ALL ))
then
    PIAXI_DENSITY_RANGE="20 30"
    PIAXI_DENSITY_N=5
    PIAXI_DENSITY_ARGS="--scan_rho ${PIAXI_DENSITY_RANGE} --scan_rho_N ${PIAXI_DENSITY_N}"
else
    INPUT_DENSITY="${1:-0.4}"
    PIAXI_DENSITY="${INPUT_DENSITY}"
    PIAXI_DENSITY_ARGS="--rho ${PIAXI_DENSITY}"
fi

# F_pi [GeV] || For QCD axion case ~ 2/g_a
if (( $SCAN_ALL ))
then
    PIAXI_F_RANGE="10 50"
    PIAXI_N_F=5
    PIAXI_F_ARGS="--scan_F ${PIAXI_F_RANGE} --scan_F_N ${PIAXI_N_F} ${PIAXI_FIT}"
else
    PIAXI_F="1e10"
    PIAXI_F_ARGS="--F ${PIAXI_F} ${PIAXI_FIT}"
fi

# m_I [eV] || For QCD axion case ~ (m_a)^2 / F_pi
if (( $SCAN_ALL ))
then
    PIAXI_N_QMASS=10
    PIAXI_MASS_RANGE="-80 -20"
    PIAXI_M_ARGS="--scan_mass ${PIAXI_MASS_RANGE} --scan_mass_N ${PIAXI_N_QMASS}"
else
    PIAXI_MASS="1e-80"
    PIAXI_M_ARGS="--m_scale ${PIAXI_MASS}"
fi

# Pi-Axiverse coupling constants: Lambda_3 and Lambda_4 [GeV]
if (( $SCAN_ALL ))
then
    if [[ "${INPUT_SCENARIO}" == "FULL" ]] || [[ "${INPUT_SCENARIO}" == "SAMPLED" ]] || [[ "${INPUT_SCENARIO}" == "SU6" ]] || [[ "${INPUT_SCENARIO}" == "CHARGED" ]] || [[ "${INPUT_SCENARIO}" == "COMPLEX" ]]
    then
        PIAXI_L3_RANGE="10 50"
        PIAXI_N_L3=5
        L3_ARGS="--scan_Lambda3 ${PIAXI_L3_RANGE} --scan_Lambda3_N ${PIAXI_N_L3}"
    else
        PIAXI_L3_RANGE="None"
        PIAXI_N_L3="None"
        L3_ARGS=""
    fi
    PIAXI_L4_RANGE="10 50"
    PIAXI_N_L4=5
    L4_ARGS="--scan_Lambda4 ${PIAXI_L4_RANGE} --scan_Lambda4_N ${PIAXI_N_L4}"
    PIAXI_L_ARGS="${L3_ARGS} ${L4_ARGS}"
else
    PIAXI_L3="1e0"
    PIAXI_L4="1e10"
    PIAXI_L_ARGS="--L3 ${PIAXI_L3} --L4 ${PIAXI_L4}"
fi

# Millicharge: epsilon
if (( $SCAN_ALL ))
then
    PIAXI_N_EPS=10
    PIAXI_EPS_RANGE="0 -20"
    PIAXI_EPS_ARGS="--scan_epsilon ${PIAXI_EPS_RANGE} --scan_epsilon_N ${PIAXI_N_EPS}"
else
    PIAXI_EPS="1e0"
    PIAXI_EPS_ARGS="--eps ${PIAXI_EPS}"
fi

# Numerical integration method ("RK45" or "BDF")
INPUT_METHOD="${7:-RK45}"
PIAXI_METHOD="${INPUT_METHOD}"

# Whether or not to perform numerical integration calculations in log-scale
INPUT_LOGSCALE="${10:-False}"
if [[ "${INPUT_LOGSCALE}" == "True" ]]
then
    PIAXI_LOGSCALE="--use_logsumexp"
else
    PIAXI_LOGSCALE=""
fi

# Pass relevant args to script
PIAXI_INPUT_ARGS="${PIAXI_SEED} --int_method ${PIAXI_METHOD} ${PIAXI_UNITS} ${PIAXI_LOGSCALE} --num_samples ${N_SAMPLES} --verbosity ${PIAXI_VERBOSITY} ${PIAXI_RES_ARGS} --config_name ${PIAXI_SYS_NAME} ${PIAXI_SAVE}"
PIAXI_PARAM_ARGS="${PIAXI_DENSITY_ARGS} ${PIAXI_F_ARGS} ${PIAXI_M_ARGS} ${PIAXI_L_ARGS} ${PIAXI_EPS_ARGS} --dqm_c ${PIAXI_DQMC} ${PIAXI_MASKS}"
PIAXI_SLURM_ARGS="--num_cores ${PIAXI_N_CORES} --num_nodes ${PIAXI_N_NODES} --job_qos ${PIAXI_JOB_QOS} --mem_per_core ${PIAXI_COREMEM}"

if [[ "${PIAXI_VERBOSITY}" -gt "3" ]]
then
    echo "INPUT ARGS: ${PIAXI_INPUT_ARGS}"
    echo "PARAM ARGS: ${PIAXI_PARAM_ARGS}"
    echo "SLURM ARGS: ${PIAXI_SLURM_ARGS}"
fi

python piaxiverse.py $PIAXI_INPUT_ARGS $PIAXI_TKRES_ARGS $PIAXI_SLURM_ARGS $PIAXI_PARAM_ARGS --no-skip_existing


     active environment : piaxiverse
    active env location : /users/sloane1/.conda/envs/piaxiverse
            shell level : 1
       user config file : /users/sloane1/.condarc
 populated config files : 
          conda version : 23.1.0
    conda-build version : not installed
         python version : 3.10.9.final.0
       virtual packages : __archspec=1=x86_64
                          __glibc=2.34=0
                          __linux=5.14.0=0
                          __unix=0=0
       base environment : /oscar/runtime/software/external/miniconda3/23.11.0  (read only)
      conda av data dir : /oscar/runtime/software/external/miniconda3/23.11.0/etc/conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cache : /o

100%|██████████| 1991/1991 [03:35<00:00,  9.25it/s] 


0:06:09.328623 elapsed on 100 cores
fit = a: 0.01   b: 0.00   c: 0.01
pcov = 
 [[ 0.00116378  0.         -0.00115602]
 [ 0.          0.          0.        ]
 [-0.00115602  0.          0.00116378]]
log-mean-squared error: 60.52
peak resonance at k = 2 corresponds to photon frequency at 1.12e+14 Hz (Infrared)
Results saved to /users/sloane1/scratch/v3.2.6/piaxi_debug_SU6
Files saved:                                                            | (Total: 9.4MiB)
  piaxi_debug_SU6_9842a365628f3c8068feb6c766df6b71ff7e875a.json         | 238.6KiB
  piaxi_debug_SU6_9842a365628f3c8068feb6c766df6b71ff7e875a.npy          | 9.1MiB
  piaxi_debug_SU6_9842a365628f3c8068feb6c766df6b71ff7e875a_funcs.pkl    | 12.5KiB
Done!
